In [190]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Muthukumaran\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
train = pd.read_excel(r'E:\INCEPTEZ\Training\Basics_python\Books_random_forest\Participants_Data\Data_Train.xlsx')

In [128]:
test = pd.read_excel(r'E:\INCEPTEZ\Training\Basics_python\Books_random_forest\Participants_Data\Data_Test.xlsx')

In [129]:
submission = pd.read_excel(r'E:\INCEPTEZ\Training\Basics_python\Books_random_forest\Participants_Data\Sample_Submission.xlsx')

In [130]:
train.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [131]:
test.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
0,The Complete Sherlock Holmes: 2 Boxes sets,Sir Arthur Conan Doyle,"Mass Market Paperback,– 1 Oct 1986",4.4 out of 5 stars,960 customer reviews,A collection of entire body of work of the She...,Short Stories (Books),"Crime, Thriller & Mystery"
1,Learn Docker - Fundamentals of Docker 18.x: Ev...,Gabriel N. Schenker,"Paperback,– Import, 26 Apr 2018",5.0 out of 5 stars,1 customer review,Enhance your software deployment workflow usin...,Operating Systems Textbooks,"Computing, Internet & Digital Media"
2,Big Girl,Danielle Steel,"Paperback,– 17 Mar 2011",5.0 out of 5 stars,4 customer reviews,"'Watch out, world. Here I come!'\nFor Victoria...",Romance (Books),Romance
3,Think Python: How to Think Like a Computer Sci...,Allen B. Downey,"Paperback,– 2016",4.1 out of 5 stars,11 customer reviews,"If you want to learn how to program, working w...",Programming & Software Development (Books),"Computing, Internet & Digital Media"
4,Oxford Word Skills: Advanced - Idioms & Phrasa...,Redman Gairns,"Paperback,– 26 Dec 2011",4.4 out of 5 stars,9 customer reviews,"Learn and practise the verbs, prepositions and...",Linguistics (Books),"Language, Linguistics & Writing"


In [132]:
submission.head()

,Price
0,119
1,123
2,108
3,118
4,132


In [133]:
train.shape , test.shape

((6237, 9), (1560, 8))

In [134]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6237 entries, 0 to 6236
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         6237 non-null   object 
 1   Author        6237 non-null   object 
 2   Edition       6237 non-null   object 
 3   Reviews       6237 non-null   object 
 4   Ratings       6237 non-null   object 
 5   Synopsis      6237 non-null   object 
 6   Genre         6237 non-null   object 
 7   BookCategory  6237 non-null   object 
 8   Price         6237 non-null   float64
dtypes: float64(1), object(8)
memory usage: 438.7+ KB


In [135]:
train.nunique() , test.nunique()

(Title           5568
 Author          3679
 Edition         3370
 Reviews           36
 Ratings          342
 Synopsis        5549
 Genre            345
 BookCategory      11
 Price           1614
 dtype: int64,
 Title           1521
 Author          1224
 Edition         1259
 Reviews           30
 Ratings          163
 Synopsis        1519
 Genre            225
 BookCategory      11
 dtype: int64)

In [136]:
train.duplicated().sum()

0

In [137]:
test.duplicated().sum()

8

### In test data, we have duplicates, we are chcing , we cant do anything about uit

In [138]:
test[test.duplicated()]

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
1090,The Rustlers of the West Fork (Hopalong Cassidy),Louis L'Amour,"Mass Market Paperback,– 1 Apr 1992",5.0 out of 5 stars,2 customer reviews,"In this first of four classic frontier novels,...",Action & Adventure (Books),Action & Adventure
1095,The Princess Bride: S. Morgenstern's Classic T...,William Goldman,"Mass Market Paperback,– 26 Mar 2010",3.8 out of 5 stars,6 customer reviews,William Goldman's modern fantasy classic is a ...,Action & Adventure (Books),Action & Adventure
1103,Everybody Lies: The New York Times Bestseller,Seth Stephens-Davidowitz,"Paperback,– 18 Apr 2018",4.3 out of 5 stars,27 customer reviews,THE NEW YORK TIMES BESTSELLER\nAN ECONOMIST BO...,Political Structure & Processes (Books),Action & Adventure
1253,Flint,Louis L'Amour,"Mass Market Paperback,– 3 Nov 1997",4.3 out of 5 stars,4 customer reviews,"He left the West at the age of seventeen, leav...",Action & Adventure (Books),Action & Adventure
1316,The Five Greatest Warriors (Jack West Series),Matthew Reilly,"Paperback,– 25 Nov 2010",4.4 out of 5 stars,12 customer reviews,The end is approaching ... Can Jack West unrav...,Action & Adventure (Books),Action & Adventure
1340,Alita: Battle Angel – The Official Movie Novel...,Pat Cadigan,"Paperback,– 14 Feb 2019",3.9 out of 5 stars,7 customer reviews,The ofﬁcial novelization to the highly anticip...,Action & Adventure (Books),Action & Adventure
1407,The Curious Incident of the Dog in the Night-t...,Mark Haddon,"Paperback,– 2 Aug 2012",4.0 out of 5 stars,96 customer reviews,A Children’s Bookshelf Selection: Each month o...,Action & Adventure (Books),Action & Adventure
1453,Storm and Silence: Volume 1 (Storm and Silence...,Robert Thier,"Paperback,– 19 Mar 2016",5.0 out of 5 stars,59 customer reviews,Freedom - that is what Lilly Linton wants most...,Action & Adventure (Books),Action & Adventure


In [139]:
test[test['Title']=='The Rustlers of the West Fork (Hopalong Cassidy)']

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
590,The Rustlers of the West Fork (Hopalong Cassidy),Louis L'Amour,"Mass Market Paperback,– 1 Apr 1992",5.0 out of 5 stars,2 customer reviews,"In this first of four classic frontier novels,...",Action & Adventure (Books),Action & Adventure
1090,The Rustlers of the West Fork (Hopalong Cassidy),Louis L'Amour,"Mass Market Paperback,– 1 Apr 1992",5.0 out of 5 stars,2 customer reviews,"In this first of four classic frontier novels,...",Action & Adventure (Books),Action & Adventure


In [140]:
test[test['Title']=='Flint']

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
1171,Flint,Louis L'Amour,"Mass Market Paperback,– 3 Nov 1997",4.3 out of 5 stars,4 customer reviews,"He left the West at the age of seventeen, leav...",Action & Adventure (Books),Action & Adventure
1253,Flint,Louis L'Amour,"Mass Market Paperback,– 3 Nov 1997",4.3 out of 5 stars,4 customer reviews,"He left the West at the age of seventeen, leav...",Action & Adventure (Books),Action & Adventure


In [141]:
train['source']='train'
test['source']='test'

In [142]:
data = pd.concat([train,test])

In [143]:
data.isna().sum()

Title              0
Author             0
Edition            0
Reviews            0
Ratings            0
Synopsis           0
Genre              0
BookCategory       0
Price           1560
source             0
dtype: int64

In [144]:
data =data.reset_index(drop=True)

In [145]:
data.tail()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source
7792,100 Things Every Designer Needs to Know About ...,Susan Weinschenk,"Paperback,– 14 Apr 2011",5.0 out of 5 stars,4 customer reviews,We design to elicit responses from people. We ...,Design,"Computing, Internet & Digital Media",NaN,test
7793,"Modern Letter Writing Course: Personal, Busine...",ARUN SAGAR,"Paperback,– 8 May 2013",3.6 out of 5 stars,13 customer reviews,"A 30-day course to write simple, sharp and att...",Children's Reference (Books),"Biographies, Diaries & True Accounts",NaN,test
7794,The Kite Runner Graphic Novel,Khaled Hosseini,"Paperback,– 6 Sep 2011",4.0 out of 5 stars,5 customer reviews,The perennial bestseller-now available as a se...,Humour (Books),Humour,NaN,test
7795,Panzer Leader (Penguin World War II Collection),Heinz Guderian,"Paperback,– 22 Sep 2009",3.5 out of 5 stars,3 customer reviews,Heinz Guderian - master of the Blitzkrieg and ...,United States History,"Biographies, Diaries & True Accounts",NaN,test
7796,Complete Spanish Step-by-Step,Barbara Bregstein,"Paperback,– 16 Sep 2016",4.5 out of 5 stars,2 customer reviews,Learn Spanish with the most convenient and eff...,Dictionaries,"Language, Linguistics & Writing",NaN,test


### Datacleaning

In [146]:
data['Reviews'] = data['Reviews'].apply(lambda x: float(x.split(' ')[0]))

In [147]:
data['Reviews'].head()

0    4.0
1    3.9
2    4.8
3    4.1
4    5.0
Name: Reviews, dtype: float64

In [148]:
data['Ratings'] =data['Ratings'].apply(lambda x:(x.split(' ')[0]))

In [149]:
data['Ratings'] =data['Ratings'].apply(lambda x: x.replace(',',''))

In [150]:
data['Ratings'] =data['Ratings'].astype(int)

In [151]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train


In [152]:
data['Publication']=data['Edition'].apply(lambda x: x.split(',')[0])

In [153]:
data['Publication_date']=data['Edition'].apply(lambda x: x.split(',')[1])

In [154]:
data['Publication'].unique()

array(['Paperback', 'Hardcover', 'Mass Market Paperback', 'Sheet music',
       'Flexibound', 'Plastic Comb', 'Loose Leaf', 'Tankobon Softcover',
       'Perfect Paperback', 'Board book', 'Cards', 'Spiral-bound',
       '(Kannada)', 'Product Bundle', 'Library Binding', '(German)',
       'Leather Bound', '(French)', '(Spanish)', '(Chinese)'],
      dtype=object)

In [155]:
data['Publication'].value_counts(dropna=False)

Paperback                6458
Hardcover                1056
Mass Market Paperback     200
Sheet music                32
Flexibound                 19
Cards                       9
Spiral-bound                6
Loose Leaf                  3
Tankobon Softcover          3
Perfect Paperback           1
Board book                  1
Plastic Comb                1
(Kannada)                   1
Product Bundle              1
Library Binding             1
(German)                    1
Leather Bound               1
(French)                    1
(Spanish)                   1
(Chinese)                   1
Name: Publication, dtype: int64

In [156]:
data['Publication'].isna().sum()

0

In [157]:
major_publication = ['Paperback', 'Hardcover']       

In [158]:
data['Publication'] = data['Publication'].apply(lambda x: x if x in major_publication else 'other')

In [159]:
data['Publication_year'] = data['Edition'].str[-4:]

In [160]:
data['Publication_year'].unique()

array(['2016', '2012', '1982', '2017', '2006', '2009', '2018', '2015',
       '2013', '1999', '2002', '2011', '1991', '2014', '1989', '2000',
       '2005', '2019', '2008', '2004', '2010', '2007', '2001', '1969',
       '1993', '1992', '2003', '1996', 'port', '1997', '1995', 'NTSC',
       '1987', '1986', '1990', '1988', '1981', '1976', '1994', '1998',
       '1977', '1974', '1983', '1971', '1985', '1978', 'mile', ' set',
       'tion', '1964', '1984', '1980', 'dged', '1979', 'rint', '1960',
       '1970', '1975', '1905', '1900', 'book', '1961', '1925', '1973',
       'ated', '1942', '1972'], dtype=object)

In [161]:
import re  ## regular expression

In [162]:
data['Publication_year'] =data['Publication_year'].apply(lambda x: re.sub('[^0-9]+', 'NA', x))

In [163]:
data['Publication_year'] = data['Publication_year'].replace('NA', 0)

In [164]:
data['Publication_year'] = data['Publication_year'].astype(int)

In [165]:
data['Age_of_book'] = 2022-data['Publication_year']

In [166]:
data['Age_of_book'].unique()

array([   6,   10,   40,    5,   16,   13,    4,    7,    9,   23,   20,
         11,   31,    8,   33,   22,   17,    3,   14,   18,   12,   15,
         21,   53,   29,   30,   19,   26, 2022,   25,   27,   35,   36,
         32,   34,   41,   46,   28,   24,   45,   48,   39,   51,   37,
         44,   58,   38,   42,   43,   62,   52,   47,  117,  122,   61,
         97,   49,   80,   50])

In [167]:
data['Age_of_book'] =data['Age_of_book'].replace(2022,0)

In [168]:
data['Age_of_book'].unique()

array([  6,  10,  40,   5,  16,  13,   4,   7,   9,  23,  20,  11,  31,
         8,  33,  22,  17,   3,  14,  18,  12,  15,  21,  53,  29,  30,
        19,  26,   0,  25,  27,  35,  36,  32,  34,  41,  46,  28,  24,
        45,  48,  39,  51,  37,  44,  58,  38,  42,  43,  62,  52,  47,
       117, 122,  61,  97,  49,  80,  50])

In [122]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_date,Publication_year,Age_of_book
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train,Paperback,– 10 Mar 2016,2016,6
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train,Paperback,– 7 Nov 2012,2012,10
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train,Paperback,– 25 Feb 1982,1982,40
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train,Paperback,– 5 Oct 2017,2017,5
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train,Hardcover,– 10 Oct 2006,2006,16


In [169]:
data['Age_of_book'] =data['Age_of_book'].replace(0,10)        ##replacing with mean for book's age

In [170]:
data['Age_of_book'].unique()

array([  6,  10,  40,   5,  16,  13,   4,   7,   9,  23,  20,  11,  31,
         8,  33,  22,  17,   3,  14,  18,  12,  15,  21,  53,  29,  30,
        19,  26,  25,  27,  35,  36,  32,  34,  41,  46,  28,  24,  45,
        48,  39,  51,  37,  44,  58,  38,  42,  43,  62,  52,  47, 117,
       122,  61,  97,  49,  80,  50])

### remove stopwords

In [193]:
stop_words = set(stopwords.words('english'))

### functionto implement nlp process for stop words

In [204]:
def pre_process_nlp(x):
    string = ''
    x =re.sub('[^A-Za-z]',' ',x)
    for word in x.lower().split():
        if not word in stop_words:
            string += word + ' '
    return string

In [209]:
data['Title'].apply(pre_process_nlp)

0                                  prisoner gold hunters 
1                           guru dutt tragedy three acts 
2                             leviathan penguin classics 
3                            pocket full rye miss marple 
4                   life years extraordinary photography 
                              ...                        
7792    things every designer needs know people voices...
7793    modern letter writing course personal business...
7794                           kite runner graphic novel 
7795       panzer leader penguin world war ii collection 
7796                          complete spanish step step 
Name: Title, Length: 7797, dtype: object

In [208]:
pre_process_nlp("The Prisoner's Gold (The Hunters 3)")

'prisoner gold hunters '

In [212]:
process_column = ['Title','Author','Edition','Synopsis','Genre','BookCategory']

In [213]:
for i in process_column:
    data[i]= data[i].apply(pre_process_nlp)

In [214]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_date,Publication_year,Age_of_book
0,prisoner gold hunters,chris kuzneski,paperback mar,4.0,8,hunters return third brilliant novel sunday ti...,action adventure books,action adventure,220.00,train,Paperback,– 10 Mar 2016,2016,6
1,guru dutt tragedy three acts,arun khopkar,paperback nov,3.9,14,layered portrait troubled genius art merely th...,cinema broadcast books,biographies diaries true accounts,202.93,train,Paperback,– 7 Nov 2012,2012,10
2,leviathan penguin classics,thomas hobbes,paperback feb,4.8,6,time men live without common power keep awe co...,international relations,humour,299.00,train,Paperback,– 25 Feb 1982,1982,40
3,pocket full rye miss marple,agatha christie,paperback oct,4.1,13,handful grain found pocket murdered businessma...,contemporary fiction books,crime thriller mystery,180.00,train,Paperback,– 5 Oct 2017,2017,5
4,life years extraordinary photography,editors life,hardcover oct,5.0,1,seven decades life thrilling world unrivalled ...,photography textbooks,arts film photography,965.62,train,Hardcover,– 10 Oct 2006,2006,16


In [215]:
data['Genre'].value_counts()

action adventure books                1183
romance books                          517
biographies autobiographies books      463
crime thriller mystery books           335
contemporary fiction books             325
                                      ... 
bodybuilding weightlifting books         1
immunology                               1
target shooting books                    1
ecotourism                               1
military history books                   1
Name: Genre, Length: 363, dtype: int64

# simple hack  -- important one

In [216]:
data['Genre'].map(data['Genre'].value_counts())

0       1183
1         92
2         25
3        325
4         47
        ... 
7792     120
7793      19
7794     119
7795       5
7796      98
Name: Genre, Length: 7797, dtype: int64

In [237]:
data['BookCategory'].map(data['BookCategory'].value_counts())

0       1036
1        732
2        670
3        878
4        638
        ... 
7792     648
7793     732
7794     670
7795     732
7796     733
Name: BookCategory, Length: 7797, dtype: int64

In [217]:
process_column = ['Title','Author','Edition','Synopsis','Genre','BookCategory']

for i in process_column:
    data[i]= data[i].map(data[i].value_counts())

In [219]:
data

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_date,Publication_year,Age_of_book
0,1,4,431,4.0,8,1,1183,1036,220.00,train,Paperback,– 10 Mar 2016,2016,6
1,2,2,446,3.9,14,2,92,732,202.93,train,Paperback,– 7 Nov 2012,2012,10
2,3,3,366,4.8,6,3,25,670,299.00,train,Paperback,– 25 Feb 1982,1982,40
3,1,87,559,4.1,13,1,325,878,180.00,train,Paperback,– 5 Oct 2017,2017,5
4,1,1,117,5.0,1,1,47,638,965.62,train,Hardcover,– 10 Oct 2006,2006,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7792,2,2,442,5.0,4,2,120,648,NaN,test,Paperback,– 14 Apr 2011,2011,11
7793,2,2,486,3.6,13,2,19,732,NaN,test,Paperback,– 8 May 2013,2013,9
7794,1,2,493,4.0,5,1,119,670,NaN,test,Paperback,– 6 Sep 2011,2011,11
7795,1,1,493,3.5,3,1,5,732,NaN,test,Paperback,– 22 Sep 2009,2009,13


In [220]:
data.columns

Index(['Title', 'Author', 'Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre',
       'BookCategory', 'Price', 'source', 'Publication', 'Publication_date',
       'Publication_year', 'Age_of_book'],
      dtype='object')

In [222]:
data['Title'].unique()

array([ 1,  2,  3, 17,  5,  4, 11, 14,  8,  7, 10,  9], dtype=int64)

In [223]:
dat_filter = data[['Title', 'Author', 'Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre',
       'BookCategory', 'Price', 'source', 'Publication', 
        'Age_of_book']]

In [224]:
dat_filter

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Age_of_book
0,1,4,431,4.0,8,1,1183,1036,220.00,train,Paperback,6
1,2,2,446,3.9,14,2,92,732,202.93,train,Paperback,10
2,3,3,366,4.8,6,3,25,670,299.00,train,Paperback,40
3,1,87,559,4.1,13,1,325,878,180.00,train,Paperback,5
4,1,1,117,5.0,1,1,47,638,965.62,train,Hardcover,16
...,...,...,...,...,...,...,...,...,...,...,...,...
7792,2,2,442,5.0,4,2,120,648,NaN,test,Paperback,11
7793,2,2,486,3.6,13,2,19,732,NaN,test,Paperback,9
7794,1,2,493,4.0,5,1,119,670,NaN,test,Paperback,11
7795,1,1,493,3.5,3,1,5,732,NaN,test,Paperback,13


In [225]:
dat_filter_category = pd.get_dummies(data[['Publication']])

In [226]:
data_preprocessed = pd.concat([dat_filter_category,dat_filter.drop(columns=['Publication'])], axis=1)

In [227]:
train_preprocessed = data_preprocessed[data_preprocessed['source']=='train']

In [228]:
test_preprocessed = data_preprocessed[data_preprocessed['source']=='test']

In [229]:
train_preprocessed.drop(columns=['source'], inplace=True)

C:\Users\Muthukumaran\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [230]:
test_preprocessed.drop(columns=['source','Price'], inplace=True)

In [231]:
test_preprocessed.reset_index(drop=True, inplace =True)

In [232]:
test_preprocessed

,Publication_Hardcover,Publication_Paperback,Publication_other,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Age_of_book
0,0,0,1,2,10,19,4.4,960,2,40,878,36
1,0,1,0,1,1,52,5.0,1,1,7,648,4
2,0,1,0,1,25,431,5.0,4,1,517,702,11
3,0,1,0,1,1,385,4.1,11,1,43,648,6
4,0,1,0,1,4,334,4.4,9,2,80,733,11
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,0,1,0,2,2,442,5.0,4,2,120,648,11
1556,0,1,0,2,2,486,3.6,13,2,19,732,9
1557,0,1,0,1,2,493,4.0,5,1,119,670,11
1558,0,1,0,1,1,493,3.5,3,1,5,732,13


In [233]:
train_preprocessed.shape , test_preprocessed.shape

((6237, 13), (1560, 12))

In [236]:
train_preprocessed.to_csv('train_preprocessed_2.csv', index= False)
test_preprocessed.to_csv('test_preprocessed_2.csv', index= False)